# 11 abr 2018

O objetivo de hoje é terminar a migração dos datasets de candidatos (resta apenas o de 2008) separadamente. Feito isso, vamos alterar o algoritmos para que possamos migrar todos os dados de uma só vez, atualizando as informações do candidato

In [5]:
import pandas as pd
import numpy as np
import mysql.connector as mysql

In [2]:
import os
import glob

from utils.tse_data_headers import get_header

In [1]:
bd_config = {
    'user': 'root',
    'password': '',
    'host': '127.0.0.1',
    'database': 'laddres'
}

In [3]:
# DIRETORIO_TEMPORARIO = mkdtemp()
DIRETORIO_TEMPORARIO = os.path.join('../temp')

# Nome e endereço dos arquivos do diretório do TSE
PREFIXO_ARQUIVO_CANDIDATURAS = 'consulta_cand_'

In [5]:
import sys
import time

import utils.tse_parsers as parsers

def migrar_dados(ano):
    tempo_inicio_ano = time.time()

    arquivos = PREFIXO_ARQUIVO_CANDIDATURAS + str(ano) + '*.txt'
    pathes = os.path.join(DIRETORIO_TEMPORARIO, arquivos)
    arquivos_do_ano = sorted(glob.glob(pathes))

    for arquivo_estado in arquivos_do_ano:
        tempo_inicio_estado = time.time()
        print('Iniciando ', arquivo_estado)

        df_estado = pd.read_csv(
            arquivo_estado,
            sep = ';',
            header = None,
            dtype = np.str,
            names = get_header(ano),
            encoding = 'iso-8859-1'
        )

        candidatos = (
            df_estado[[
                'NOME_CANDIDATO',
                'DATA_NASCIMENTO',
                'CPF_CANDIDATO',
                'NUM_TITULO_ELEITORAL_CANDIDATO',
#                 'NM_EMAIL',
                'NOME_MUNICIPIO_NASCIMENTO',
                'SIGLA_UF_NASCIMENTO',
                'CODIGO_NACIONALIDADE',
                'COD_GRAU_INSTRUCAO',
                'CODIGO_OCUPACAO',
            ]]
            .drop_duplicates('CPF_CANDIDATO', keep='last')
            .rename(columns = {
                'NOME_CANDIDATO': 'nome',
                'DATA_NASCIMENTO': 'data_nascimento',
                'CPF_CANDIDATO': 'cpf',
                'NUM_TITULO_ELEITORAL_CANDIDATO': 'titulo_eleitoral',
#                 'NM_EMAIL': 'email',
                'NOME_MUNICIPIO_NASCIMENTO': 'cidade',
                'SIGLA_UF_NASCIMENTO': 'estado',
                'CODIGO_NACIONALIDADE': 'nacionalidade',
                'COD_GRAU_INSTRUCAO': 'grau_instrucao',
                'CODIGO_OCUPACAO': 'ocupacao'
            })
        )

        total_candidatos = candidatos.shape[0]
        print('Total de candidatos em', ano, ':', total_candidatos)


        cnx = mysql.connect(**bd_config)
        cursor = cnx.cursor(buffered=True)

        for candidato in candidatos.itertuples():
            # Buscar id da cidade
            cidade = parsers.parse_cidade(candidato.cidade)
            sigla_estado = parsers.parse_estado(candidato.estado)
            try:
                query = ("SELECT id FROM cidade WHERE "
                     "nome = \"" + cidade + "\" AND "
                     "estado_id = (SELECT id FROM estado WHERE sigla = '" + sigla_estado + "')")
            except:
                print('Candidato: {}'.format(candidato))
                print('Cidade: {}'.format(cidade))
                print('Sigla do Estado: {}'.format(sigla_estado))
                cursor.close()
                cnx.close()
                raise

            try:
                cursor.execute(query)
            except:
                print(candidato)
                print(query)
                cursor.close()
                cnx.close()
                raise

            cidades = cursor.fetchall()
            cidade_id = cidades[0][0] if len(cidades) > 0 else None

            # Cadastrar candidato
            novo_candidato = ("INSERT INTO candidato" + str(ano) + " "
                           "(nome, data_nascimento, cpf, titulo_eleitoral, email, cidade_id, ocupacao_id, nacionalidade_id, grau_instrucao_id) "
                           "VALUES (%(nome)s, %(data_nascimento)s, %(cpf)s, %(titulo_eleitoral)s, %(email)s, %(cidade_id)s, %(ocupacao)s, %(nacionalidade)s, %(grau_instrucao)s)")
            atributos = {
                'nome': parsers.parse_nome(candidato.nome),
                'data_nascimento': parsers.parse_data(candidato.data_nascimento),
                'cpf': parsers.parse_cpf(candidato.cpf),
                'titulo_eleitoral': parsers.parse_titulo_eleitoral(candidato.titulo_eleitoral),
                'email': None,
#                 'email': parsers.parse_email(candidato.email),
                'cidade_id': cidade_id,
                'ocupacao': parsers.parse_ocupacao(candidato.ocupacao),
                'nacionalidade': parsers.parse_nacionalidade(candidato.nacionalidade),
                'grau_instrucao': parsers.parse_grau_instrucao(candidato.grau_instrucao)
            }

            try:
                cursor.execute(novo_candidato, atributos)
                cnx.commit()
            except:
                print(candidato)
                cursor.close()
                cnx.close()
                raise

            sys.stdout.write('\r{} de {}'.format(candidato[0], total_candidatos))
            sys.stdout.flush()

        cursor.close()
        cnx.close()

        sys.stdout.write('\n')
        tempo_final_estado = time.time()
        print('Tempo total desse estado: {:.2f}s \n'.format((tempo_final_estado - tempo_inicio_estado)))

    tempo_final_ano = time.time()
    print('Tempo total em {}: {:.2f}s \n'.format(ano, (tempo_final_ano - tempo_inicio_ano)))

In [6]:
migrar_dados(2008)

Iniciando  ../temp\consulta_cand_2008_PR.txt
Total de candidatos em 2008 : 26051
26158 de 26051
Tempo total desse estado: 101.58s 

Iniciando  ../temp\consulta_cand_2008_RJ.txt
Total de candidatos em 2008 : 15314
15371 de 15314
Tempo total desse estado: 56.58s 

Iniciando  ../temp\consulta_cand_2008_RN.txt
Total de candidatos em 2008 : 7169
7192 de 7169
Tempo total desse estado: 26.92s 

Iniciando  ../temp\consulta_cand_2008_RO.txt
Total de candidatos em 2008 : 4588
4605 de 4588
Tempo total desse estado: 18.11s 

Iniciando  ../temp\consulta_cand_2008_RR.txt
Total de candidatos em 2008 : 1293
1308 de 1293
Tempo total desse estado: 4.83s 

Iniciando  ../temp\consulta_cand_2008_RS.txt
Total de candidatos em 2008 : 23400
23481 de 23400
Tempo total desse estado: 96.09s 

Iniciando  ../temp\consulta_cand_2008_SC.txt
Total de candidatos em 2008 : 14340
14392 de 14340
Tempo total desse estado: 56.79s 

Iniciando  ../temp\consulta_cand_2008_SE.txt
Total de candidatos em 2008 : 4687
4710 de 4687

In [6]:
df = pd.read_csv(
    '../data/2018-04-04-candidaturas-tse.xz',
    low_memory=False
)

In [40]:
(df[['ANO_ELEICAO', 'DATA_NASCIMENTO', 'CPF_CANDIDATO', 'NUM_TITULO_ELEITORAL_CANDIDATO']]
#      .loc[df['CPF_CANDIDATO', 'NUM_TITULO_ELEITORAL_CANDIDATO'] == '#NULO#']
     .drop_duplicates(['CPF_CANDIDATO', 'NUM_TITULO_ELEITORAL_CANDIDATO'], keep='last')
     .sort_values(by=['CPF_CANDIDATO', 'NUM_TITULO_ELEITORAL_CANDIDATO'], ascending=True, na_position='first')
)
    

# ['ANO_ELEICAO', 'DATA_NASCIMENTO', 'CPF_CANDIDATO']
# df[(df['CPF_CANDIDATO'] != '#NULO#') & (df['CODIGO_CARGO'] != 91)]

,ANO_ELEICAO,DATA_NASCIMENTO,CPF_CANDIDATO,NUM_TITULO_ELEITORAL_CANDIDATO
390396,2004,NaN,#NULO#,#NI#
800173,2008,01-OCT-83,#NULO#,000000000055
800172,2008,01-OCT-83,#NULO#,000000000077
282287,2004,29091963,#NULO#,000069042631
282225,2004,28091947,#NULO#,000094342607
1345,2004,08101956,#NULO#,000264232470
282193,2004,21061951,#NULO#,000270002682
282669,2004,11021955,#NULO#,000288772623
282664,2004,19101942,#NULO#,000300262682
282167,2004,22031960,#NULO#,000320832682


---

In [1]:
import utils.tse_migrar_dados_por_estado as mdpe

In [2]:
mdpe.migrar_dados_candidato(2006)

Iniciando ../temp\consulta_cand_2006_AC.txt
Total de candidatos em 2006: 462
463 de 462
Tempo total desse estado: 1.88s 

Iniciando ../temp\consulta_cand_2006_AL.txt
Total de candidatos em 2006: 393
395 de 393
Tempo total desse estado: 1.91s 

Iniciando ../temp\consulta_cand_2006_AM.txt
Total de candidatos em 2006: 561
560 de 561
Tempo total desse estado: 2.43s 

Iniciando ../temp\consulta_cand_2006_AP.txt
Total de candidatos em 2006: 386
385 de 386
Tempo total desse estado: 1.74s 

Iniciando ../temp\consulta_cand_2006_BA.txt
Total de candidatos em 2006: 884
888 de 884
Tempo total desse estado: 4.46s 

Iniciando ../temp\consulta_cand_2006_BR.txt
Total de candidatos em 2006: 16
19 de 16
Tempo total desse estado: 0.10s 

Iniciando ../temp\consulta_cand_2006_CE.txt
Total de candidatos em 2006: 785
787 de 785
Tempo total desse estado: 3.99s 

Iniciando ../temp\consulta_cand_2006_DF.txt
Total de candidatos em 2006: 863
869 de 863
Tempo total desse estado: 4.75s 

Iniciando ../temp\consulta_

In [3]:
mdpe.migrar_dados_candidato(2010)

Iniciando ../temp\consulta_cand_2010_AC.txt
Total de candidatos em 2010: 455
455 de 455
Tempo total desse estado: 5.23s 

Iniciando ../temp\consulta_cand_2010_AL.txt
Total de candidatos em 2010: 463
470 de 463
Tempo total desse estado: 5.44s 

Iniciando ../temp\consulta_cand_2010_AM.txt
Total de candidatos em 2010: 533
534 de 533
Tempo total desse estado: 6.30s 

Iniciando ../temp\consulta_cand_2010_AP.txt
Total de candidatos em 2010: 399
486 de 399
Tempo total desse estado: 4.80s 

Iniciando ../temp\consulta_cand_2010_BA.txt
Total de candidatos em 2010: 1013
1019 de 1013
Tempo total desse estado: 12.52s 

Iniciando ../temp\consulta_cand_2010_BR.txt
Total de candidatos em 2010: 18
21 de 18
Tempo total desse estado: 0.24s 

Iniciando ../temp\consulta_cand_2010_CE.txt
Total de candidatos em 2010: 799
806 de 799
Tempo total desse estado: 9.93s 

Iniciando ../temp\consulta_cand_2010_DF.txt
Total de candidatos em 2010: 1058
1065 de 1058
Tempo total desse estado: 13.45s 

Iniciando ../temp\c

In [2]:
mdpe.migrar_dados_candidato(2014)

Iniciando ../temp\consulta_cand_2014_BR.txt
Total de candidatos em 2014: 23
27 de 23
Tempo total desse estado: 0.59s 

Tempo total em 2014: 0.59s 



---

## Candidaturas

In [9]:
(df[['NUMERO_PARTIDO']]
#      .loc[df['CPF_CANDIDATO', 'NUM_TITULO_ELEITORAL_CANDIDATO'] == '#NULO#']
     .drop_duplicates(['NUMERO_PARTIDO'], keep='last')
     .sort_values(by=['NUMERO_PARTIDO'], ascending=True, na_position='first')
)

,NUMERO_PARTIDO
1837240,10
1837255,11
1837229,12
1837253,13
1837256,14
1837257,15
1824733,16
1837217,17
1836945,18
1837140,19
